## Making MDPs in Python

For the simple environments, we can just hardcode the MDPs into a dictionary by exhaustively encoding the whole state space and the transition function. We will also go through a more complicated example where the state space is too large to be manually coded and we need to implement the transition function based on some state parameters.

You can start this assignment during/after reading Grokking Ch-2.

NOTE: all page numbers referred to in this ipynb notebook refer to Grokking textbook

## Environment 0 - Bandit Walk

Let us consider the BW environment on Page 39. 

State Space has 3 elements, states 0, 1 and 2.
States 0 and 2 are terminal states and state 1 is the starting state.

Action space has 2 elements, left and right.

The environment is deterministic - transition probability of any action is 1.

Only 1 (State, Action, State') tuple has positive reward, (1, Right, 2) gives the agent +1 reward.

We'll model this MDP as a dictionary. This code is an example for the upcoming exercises.

In [2]:
bw_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)]
    },

    1 : {
        "Right" : [(1, 2, 1, True)],
        "Left" : [(1, 0, 0, True)]
    },

    2 : {
        "Right" : [(1, 2, 0, True)],
        "Left" : [(1, 2, 0, True)]
    }
    
}

Here we a dictionary bw_mdp which to get the transitions possible we can do bw_mdp\[current state\]\[action to take\]. this will return a list of the possibilities.

The format of each element within the list is   
(probability of that transition , next state , reward , 'is the next state a terminal state?')

Obviously make sure that the sum of probability of all possible transitions is 1

Note that by convention, all actions from terminal states still lead to the same state with reward 0.

## Environment 1 - Slippery Walk

Here, states 0 and 6 are terminal states and state 3 is the starting state.

Action space has again 2 elements, left and right.

The environment is now stochastic, transition probability of any action is as follows -
If agent chooses `Right` at a non-terminal state,
- $50\%$ times it will go to `Right` state
- $33\frac{1}{3} \%$ times it will stay in same state
- $16\frac{2}{3}\%$ times it will go to `Left`state

And similiarly vice versa for Left(just replace left with right and right with left in previous list)

This time, 2 different (State, Action, State') tuples have positive rewards, you need to find them.

We'll again model this MDP as a dictionary. Part of the code is written for you.

In [3]:
swf_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)],
    },

    1 : {
        "Right" : [
            (1/2, 2, 1, False),
            (1/3, 1, 0, False),
            (1/6, 0, 1, True),
        ],
        "Left" : [
            (1/2, 0, 1, True),
            (1/3, 1, 0, False),
            (1/6, 2, 1, False)
        ]
    }, 

    2 : {
        "Right" : [
            (1/2, 3, 1, False),
            (1/3, 2, 0, False),
            (1/6, 1, 1, False),
        ],
        "Left" : [
            (1/2, 1, 1, False),
            (1/3, 2, 0, False),
            (1/6, 3, 1, False)
        ]
    },
    
    3 : {
        "Right" : [
            (1/2, 4, 1, False),
            (1/3, 3, 0, False),
            (1/6, 2, 1, False),
        ],
        "Left" : [
            (1/2, 2, 1, False),
            (1/3, 3, 0, False),
            (1/6, 4, 1, False)
        ]
    },
    
    4 : {
        "Right" : [
            (1/2, 5, 1, False),
            (1/3, 4, 0, False),
            (1/6, 3, 1, False),
        ],
        "Left" : [
            (1/2, 3, 1, False),
            (1/3, 4, 0, False),
            (1/6, 5, 1, False)
        ]
    },
    
    5 : {
        "Right" : [
            (1/2, 6, 1, True),
            (1/3, 5, 0, False),
            (1/6, 4, 1, False),
        ],
        "Left" : [
            (1/2, 4, 1, False),
            (1/3, 5, 0, False),
            (1/6, 6, 1, True)
        ]
    },
    
    6 : {
        "Right" : [
            (1, 6, 0, True)
        ],
        "Left" : [
            (1, 6, 0, True)
        ]
    },
    
}

## Environment 2 - Frozen Lake Environment

This environment is described on Page 46.

The FL environment has a large state space, so it's better to generate most of the MDP via Python instead of typing stuff manually.

Note that all 5 states - 5, 7, 11, 12, 15 are terminal states, so keep that in mind while constructing the MDP.

There are 4 actions now - Up, Down, Left, Right.

The environment is stochastic, and states at the border of lake will require separate treatment.



Yet again we will model this MDP as a (large) dictionary.

In [4]:
fl_mdp = {
    i: {} for i in range(16)
}

In [5]:
for state in range(0, 16):

    # add transitions to other states
    transitions = {}
    holes = [5,7,11,12, 15]
    goal = 15
    for action in ["Up", "Down", "Right", "Left"]:
        transitions[action] = []
        if state not in holes:
            nexs = []
            st = state
            if (st+1)%4 > st%4 and st+1<16:
                nexs.append(st+1)
            if (st-1)%4 < st%4 and st-1 >=0:
                nexs.append(st-1)
            if (st+4)<16:
                nexs.append(st+4)
            if (st+4)<16:
                nexs.append(st-4)
            if action[0]=='U':
                if st+4 in nexs:
                    nexs.remove(st+4)
            elif action[1]=='D':
                if st-4 in nexs:
                    nexs.remove(st-4)
            elif action[0]=='R':
                if st+1 in nexs:
                    nexs.remove(st+1)
            else:
                if st-1 in nexs:
                    nexs.remove(st-1)
            for nex in nexs:
                transitions[action].append((1/3, nex if (nex>=0 and nex<16) else state, 1 if nex==goal else 0, nex in holes))
            transitions[action].append((1-len(nexs)*(1/3), state, 0, False))
            
        else:
            transitions[action].append((1, state, 0, True))
        

    fl_mdp[state] = transitions

You might need to do some stuff manually, but make sure to automate most of it.

You can check your implementation of the FL environment by comparing it with the one in OpenAI Gym.
OpenAI Gym basically contains implmentations of common RL MDPs so that we can directly train our agents on test environments. 

In [6]:
import gym
P = gym.make('FrozenLake-v1').env.P

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


Since the imported MDP is also just a dictionary, we can just print it.

In [7]:
# using the pretty print module

import pprint
pprint.pprint(P)

{0: {0: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 4, 0.0, False)],
     1: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 4, 0.0, False),
         (0.3333333333333333, 1, 0.0, False)],
     2: [(0.3333333333333333, 4, 0.0, False),
         (0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 0, 0.0, False)],
     3: [(0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
         (0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 5, 0.0, True)],
     1: [(0.3333333333333333, 0, 0.0, False),
         (0.3333333333333333, 5, 0.0, True),
         (0.3333333333333333, 2, 0.0, False)],
     2: [(0.3333333333333333, 5, 0.0, True),
         (0.3333333333333333, 2, 0.0, False),
         (0.3333333333333333, 1, 0.0, False)],
     3: [(0.3333333333333333,

In [8]:
pprint.pprint(fl_mdp)

{0: {'Down': [(0.3333333333333333, 1, 0, False),
              (0.3333333333333333, 4, 0, False),
              (0.3333333333333333, 0, 0, False),
              (0.0, 0, 0, False)],
     'Left': [(0.3333333333333333, 1, 0, False),
              (0.3333333333333333, 4, 0, False),
              (0.3333333333333333, 0, 0, False),
              (0.0, 0, 0, False)],
     'Right': [(0.3333333333333333, 4, 0, False),
               (0.3333333333333333, 0, 0, False),
               (0.33333333333333337, 0, 0, False)],
     'Up': [(0.3333333333333333, 1, 0, False),
            (0.3333333333333333, 0, 0, False),
            (0.33333333333333337, 0, 0, False)]},
 1: {'Down': [(0.3333333333333333, 2, 0, False),
              (0.3333333333333333, 5, 0, True),
              (0.3333333333333333, 1, 0, False),
              (0.0, 1, 0, False)],
     'Left': [(0.3333333333333333, 2, 0, False),
              (0.3333333333333333, 5, 0, True),
              (0.3333333333333333, 1, 0, False),
             

## Environment 3 - Key Door Game

Now we will implement a slightly different version of frozen lake without the stochasticity but with the added constraint of a key.

So now we have a 4x4 grid in which some cells are holes(and so terminal) and we have a goal state(keep it as 15) but this time someone locked the goal state with a door :(. Without the key we CANNOT ENTER the goal state.

Luckily for us, that guy was very clumsy and dropped the key in tile 3!. So now make a MDP to represent the process of us escaping this grid world.

Remember that the state in a MDP isnt just the position/location we are in. it should represent our whole game situation. (For example I can either be at tile 14 with the key or at tile 14 without key and these two situations are different and therfore must be represented as **different** states!)

To summarise
- state state is 0
- door/goal state is 15
- holes at 5, 7, 11, 12
- key at 3
- The key is picked up automatically upon entering tile 3
- can only enter tile 15 when key is obtained(otherwise its like a wall)
- holes & goal are terminal
- reward of 1 when reaching tile 15 wiht the key
- Attempting to enter tile 15 when we DONT have the key should be treated like hitting a wall and bouncing back to the original state where we took that action

In [9]:
# You can implements any helper function or other variables to help you implement it
from itertools import product
states = list(product(range(16), range(2)))
gw_mdp = {
    i: {} for i in states
}

# State is a tuple (cell_id, key), key = 1 if key is with us else key = 0

for state in states:
    acc = state[1]==1 or state[0] == 3
    # add transitions to other states
    transitions = {}
    holes = [5,7,11,12, 15]
    goal = 15
    key = 3
    for action in ["Up", "Down", "Right", "Left"]:

        transitions[action] = []
        st = state[0]
        if state not in holes:
            nexs = []
            if (st+1)%4 > st%4 and st+1<16:
                nexs.append(st+1)
            if (st-1)%4 < st%4 and st-1 >=0:
                nexs.append(st-1)
            if (st+4)<16:
                nexs.append(st+4)
            if (st+4)<16:
                nexs.append(st-4)
            if action[0]=='U':
                if st+4 in nexs:
                    nexs.remove(st+4)
            elif action[1]=='D':
                if st-4 in nexs:
                    nexs.remove(st-4)
            elif action[0]=='R':
                if st+1 in nexs:
                    nexs.remove(st+1)
            else:
                if st-1 in nexs:
                    nexs.remove(st-1)
            for j in range(len(nexs)):
                if nexs[j]==goal and acc==False:
                    nexs[j]=state[0]
                
            for nex in nexs:
                transitions[action].append((1/3, (nex if ((nex>=0 and nex<16) and (nex!=15 or acc==True)) else state[0], acc), 1 if (nex==goal and acc) else 0, nex in holes))
            transitions[action].append((1-len(nexs)*(1/3), state, 0, False))
        else:
            transitions[action].append((1, (st, acc), 0, True))
        

    gw_mdp[state] = transitions
pprint.pprint(gw_mdp)

{(0, 0): {'Down': [(0.3333333333333333, (1, False), 0, False),
                   (0.3333333333333333, (4, False), 0, False),
                   (0.3333333333333333, (0, False), 0, False),
                   (0.0, (0, 0), 0, False)],
          'Left': [(0.3333333333333333, (1, False), 0, False),
                   (0.3333333333333333, (4, False), 0, False),
                   (0.3333333333333333, (0, False), 0, False),
                   (0.0, (0, 0), 0, False)],
          'Right': [(0.3333333333333333, (4, False), 0, False),
                    (0.3333333333333333, (0, False), 0, False),
                    (0.33333333333333337, (0, 0), 0, False)],
          'Up': [(0.3333333333333333, (1, False), 0, False),
                 (0.3333333333333333, (0, False), 0, False),
                 (0.33333333333333337, (0, 0), 0, False)]},
 (0, 1): {'Down': [(0.3333333333333333, (1, True), 0, False),
                   (0.3333333333333333, (4, True), 0, False),
                   (0.33333333333333

The exact way you represent each state and each action is up to you! But... do atleast write down in a comment or something how you represent the states and action in the MDP. So i don't have to spend time deciphering.

Note: For additional challenges
- Make this grid-world also stochastic like Frozen Lake
- Make a function that allows you to input in the locations of the holes, the door(goal state) and the key and using that it will generate and return the MDP

## Environment 4 - Lights Out

**Lights out** is a deterministic puzzle game which consists of a 5x5 grid of lights which can either be on or off.

When the game begins a random set of these lights will be switched on. Pressing any light will toggle it (toggle means if the light was on then it will become off and if it was off then it will become on) and it will also toggle all of its neighbours.

![image of the game](./images/LightsOutIllustration.png)

Note: so any non-edge and non-corner cell action will toggle 5 lights, an edge cell action would toggle 4 lights and a corner cell action only toggles 3 lights.


The goal of the game is to find a sequence of moves to switch off all the lights. Now i'm not gonna make you solve this instead we are going to represent this as a deterministic MDP, where 
- the states should represent the game grid layout 
- the actions are the cells you wish to toggle
- Give a reward of 1 when solved 
- the ONLY terminal state is the fully solved state

Since 5x5 is pretty big we will restrict ourselves to a **4x4** grid and represent it as an MDP instead

In [10]:
# You can implements any helper function or other variables to help you implement it

n = 4

states = [[0 for i in range(n*n)] for _ in range(2**(n*n))] # State is a list of the grid with 1 whenever the cell is lighted

for i in range(2**(n*n)):
    x = i
    ind = (n*n)-1
    while x!=0:
        states[i][ind]=x%2
        ind-=1
        x=x//2


lo_map = {
    tuple(i):{} for i in states
}

actions = [i for i in range(n*n)]
goal = states[0]

for state in states:
    transitions = {}
    if state == goal:
        for action in actions:
            transitions[action]=[(1, state, 0, True)]
        lo_map[tuple(state)]=transitions
        continue
    for action in actions:
        togs = [action]
        
        if (action+1)%n > action%n and action+1<n*n:
            togs.append(action+1)
        if (action-1)%n < action%n and action-1>=0:
            togs.append(action-1)
        if (action+n)<n*n:
            togs.append(action+n)
        if (action-n)>=0:
            togs.append(action-n)
        nex_st = state.copy()
        for j in togs:
            nex_st[j]=(nex_st[j]+1)%2
        rew = 0
        if nex_st == goal:
            rew = 1
        transitions[action]=[(1, nex_st, rew, rew==1)] #(probability, next state, reward, is next state terminal)
    lo_map[tuple(state)]=transitions

The exact way you represent each state and each action is up to you! But... do atleast write down in a comment or something how you represent the states and action in the MDP. So i don't have to spend time deciphering.

Note: if you want you can also try to represent the 5x5 one but this is optional